In [7]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
#import evaluate
import pandas as pd
import numpy as np
dataset_path = "data/ASDiv_clean_formatted.json"

# Load the dataset from a local file or directory
dataset = load_dataset("json", data_files=dataset_path)

#Load the model
model_name='tiiuae/falcon-7b-instruct'

original_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Prepare the data 
def tokenize_function(example):
    start_prompt = (f"Below is an instruction that describes a task. "
            f"Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n")
    end_prompt = (f"\n\n### Response: ")
    prompt = [start_prompt + instruction + end_prompt for instruction in example["instruction"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["output"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['instruction', 'input', 'output',])

Found cached dataset json (/Users/bryan/.cache/huggingface/datasets/json/default-1adc5a646aa09bfa/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
pip install einops

     |████████████████████████████████| 42 kB 1.5 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.
